In [6]:
import pandas as pd
import plotly.express as px

In [8]:
df = pd.read_csv("aave_price_data.csv")
fig = px.line(df, x="Date", y="Price")

# template="plotly_dark"

fig.update_layout(
    xaxis = dict(
        tickmode = "array",
        tickvals = ['Mar 01, 2022', 'Feb 05, 2022', 'Jan 06, 2022', 'Dec 01, 2021',
                    'Nov 01, 2021', 'Oct 02, 2021', 'Sep 02, 2021', 'Aug 03, 2021',
                    'Jul 04, 2021', 'Jun 04, 2021'],
        ticktext = ['Mar 2022', 'Feb 2022', 'Jan 2022', 'Dec 2021',
                    'Nov 2021', 'Oct 2021', 'Sep 2021', 'Aug 2021',
                    'July 2021', 'June 2021']
    )
)

# # chart line color red
# fig.update_traces(line_color='#FF0000')

fig.show()

FileNotFoundError: [Errno 2] No such file or directory: 'aave_price_data.csv'

In [ ]:
df = pd.read_csv("uniswap_price_data.csv")
fig = px.line(df, x="Date", y="Price")

# template="plotly_dark"

fig.update_layout(
    xaxis = dict(
        tickmode = "array",
        tickvals = ['Mar 01, 2022', 'Feb 05, 2022', 'Jan 06, 2022', 'Dec 01, 2021',
                    'Nov 01, 2021', 'Oct 02, 2021', 'Sep 02, 2021', 'Aug 03, 2021',
                    'Jul 04, 2021', 'Jun 04, 2021'],
        ticktext = ['Mar 2022', 'Feb 2022', 'Jan 2022', 'Dec 2021',
                    'Nov 2021', 'Oct 2021', 'Sep 2021', 'Aug 2021',
                    'July 2021', 'June 2021']
    )
)

# # chart line color red
# fig.update_traces(line_color='#FF0000')

fig.show()

In [ ]:
%pip install dash

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: brotli in c:\programdata\anaconda3\lib\site-packages (from flask-compress->dash) (1.0.9)



In [ ]:
%pip install dash_bootstrap_components

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from distutils.command.build import build
from dash import Dash, html, dcc, Output, Input, callback
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
# for sentiment analysis
import numpy as np
import re
from nltk.corpus import stopwords
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
def sentiment_analysis(chosen_defi_coin):
    stop = stopwords.words('english')
    def clean_content(contentInput):
        if type(contentInput) == np.float:
            return ""
        contentInput = contentInput.lower()
        contentInput = re.sub("'", "", contentInput) # to avoid removing contractions in english
        contentInput = re.sub("@[A-Za-z0-9_]+","", contentInput)
        contentInput = re.sub("#[A-Za-z0-9_]+","", contentInput)
        contentInput = re.sub(r'http\S+', '', contentInput)
        contentInput = re.sub('[()!?]', ' ', contentInput)
        contentInput = re.sub('\[.*?\]',' ', contentInput)
        contentInput = re.sub("[^a-z0-9]"," ", contentInput)
        contentInput = contentInput.strip()
        contentInput = contentInput.split()
        contentInput = [w for w in contentInput if not w in stop]
        contentInput = " ".join(word for word in contentInput)
        return contentInput
    string = chosen_defi_coin.lower() + ".csv"
    df = pd.read_csv(string)

    df['date'] = df['created'].apply(lambda x: x.split(' ')[0])
    df.sort_values(['date'], inplace=True)

    df['year_month'] = df['date'].apply(lambda x: x[0:x.rfind('-')])

    df = df.fillna(method='ffill')
    # monthList = pd.date_range('2021-01-01','2022-02-01', 
    #             freq='MS').strftime("%m/%Y").tolist()
    monthList = list(df['year_month'].unique())

    timeSplitData={}
    timeSplitDataSentiment = {}

    for month in monthList: 
        timeSplitData[month] = []
        timeSplitDataSentiment[month] = []  

    for row in df.iterrows():
        month = row[1]['year_month']
        content = row[1]['body']
        cleaned = clean_content(content)
        timeSplitData[month].append(cleaned)

    sid = SentimentIntensityAnalyzer()

    for month in monthList: # for month in timeSplitData:
        for content in timeSplitData[month]:
            output = sid.polarity_scores(content)
            timeSplitDataSentiment[month].append(output)
    
    timeSplitDataNetScore = {}
    for month in monthList:
        timeSplitDataNetScore[month] = 0 
        for output in timeSplitDataSentiment[month]:
            compound = output['compound']
            compound = compound / len(timeSplitDataSentiment[month])
            timeSplitDataNetScore[month] +=compound

    return timeSplitDataNetScore

    df = pd.DataFrame({'date': timeSplitDataNetScore.keys(), 'sentiment': timeSplitDataNetScore.values()})
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['date'], y=df['sentiment']))
    return fig

In [ ]:
sentiment_analysis("maker")

C:\Users\caleb\AppData\Local\Temp/ipykernel_13124/193915943.py:4: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



{'2021-01': 0.3987874125874128,
 '2021-02': 0.5084216981132077,
 '2021-03': 0.49451428571428574,
 '2021-04': 0.6425643749999994,
 '2021-05': 0.46996713615023467,
 '2021-06': 0.5057185840707964,
 '2021-07': 0.6883381578947367,
 '2021-08': 0.6565422018348626,
 '2021-09': 0.5430939393939398,
 '2021-10': 0.5446525773195875,
 '2021-11': 0.6926254237288133,
 '2021-12': 0.6965793103448277}

In [ ]:
from plotly.subplots import make_subplots

In [ ]:
price_df = pd.read_csv("curve_price_data.csv")
# timeSplitDataNetScore = sentiment_analysis("curve")
sentiment_df = pd.DataFrame({'date': timeSplitDataNetScore.keys(), 'sentiment': timeSplitDataNetScore.values()})

def mdy_to_ymd(d):
    return datetime.strptime(d, '%b %d, %Y').strftime('%Y-%m-%d')

price_df['format_date'] = price_df['Date'].apply(lambda x:  mdy_to_ymd(x))

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=price_df['format_date'], y=price_df['Price'], name='Price'), secondary_y= False)
fig.update_xaxes(autorange="reversed") 
# fig.update_xaxes(dtick="M1")

fig.add_trace(go.Scatter(x=sentiment_df['date'], y=sentiment_df['sentiment'], name='Sentiment'), secondary_y= True)

fig.update_layout(
    title_text="Curve Data Price against Sentiment",
    title_x=0.5
)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Price", secondary_y=False)
fig.update_yaxes(title_text="Sentiment", secondary_y=True)

NameError: name 'timeSplitDataNetScore' is not defined

In [ ]:
price_df['format_date'].unique()

KeyError: 'format_date'

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=price_df['Date'], y=price_df['Price'], name='Price'))
fig.update_xaxes(autorange="reversed") 

In [ ]:
date = {'2021-01': 'Jan 01, 2021',
'2021-02': 'Feb 01, 2021',
'2021-03': 'Mar 01, 2021',
'2021-04': 'Apr 01, 2021',
'2021-05': 'May 01, 2021',
'2021-06': 'Jun 01, 2021',
'2021-07': 'Jul 01, 2021',
'2021-08': 'Aug 01, 2021',
'2021-09': 'Sep 01, 2021',
'2021-10': 'Oct 01, 2021',
'2021-11': 'Nov 01, 2021',
'2021-12': 'Dec 01, 2021',}

In [ ]:
import plotly.express as px
df = px.data.stocks()
fig = px.line(df, x="date", y=df.columns,
            #   hover_data={"date": "|%B %d, %Y"},
              title='custom tick labels')
fig.update_xaxes(
    dtick="M1")
    # tickformat="%b\n%Y")
fig.show()

In [ ]:
df

,date,GOOG,AAPL,AMZN,FB,NFLX,MSFT
0,2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2,2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
3,2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
4,2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708
...,...,...,...,...,...,...,...
100,2019-12-02,1.216280,1.546914,1.425061,1.075997,1.463641,1.720717
101,2019-12-09,1.222821,1.572286,1.432660,1.038855,1.421496,1.752239
102,2019-12-16,1.224418,1.596800,1.453455,1.104094,1.604362,1.784896
103,2019-12-23,1.226504,1.656000,1.521226,1.113728,1.567170,1.802472


In [ ]:
from datetime import datetime


def mdy_to_ymd(d):
    return datetime.strptime(d, '%b %d, %Y').strftime('%Y-%m-%d')

price_df['format_date'] = price_df['Date'].apply(lambda x:  mdy_to_ymd(x)[0: mdy_to_ymd(x).rfind('-')])

mdy_to_ymd('Jan 01, 2021')
# price_df.head(5)

NameError: name 'price_df' is not defined

In [ ]:
sentiment_df.head(5)

NameError: name 'sentiment_df' is not defined

In [ ]:
%pip install dash_daq

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import json
with open('sentimentalOutput/discord.json') as json_file:
    data = json.load(json_file)

FileNotFoundError: [Errno 2] No such file or directory: 'sentimentalOutput/discord.json'

In [20]:
import pandas as pd
## Group 1
grp1= ['AAVE','COMPOUND','SUSHI','UNISWAP']
grp1= ['aave','compound','sushi','uniswap']
grp2= ['CURVE']


aave = pd.read_csv('priceData/aave_price_data.csv')
compound = pd.read_csv('priceData/compound_price_data.csv')
sushi = pd.read_csv('priceData/sushi_price_data.csv')
uniswap = pd.read_csv('priceData/uniswap_price_data.csv')

date = aave['Date']
aave = aave['Price']
compound = compound['Price']
sushi = sushi['Price']
uniswap = uniswap['Price']
newdate = []

for i in date:
    newdate.append(mdy_to_ymd(i))
d = {'date':newdate,'aave':aave,'compound':compound,'sushi':sushi,'uniswap':uniswap}

df = pd.DataFrame(data=d)

In [21]:
df

,date,aave,compound,sushi,uniswap
0,2022-04-01,243.86,157.70,4.489,11.7700
1,2022-03-31,206.73,149.20,4.403,11.2900
2,2022-03-30,222.96,157.90,4.464,11.7000
3,2022-03-29,223.03,147.30,4.220,11.2500
4,2022-03-28,167.53,122.40,3.807,10.9000
...,...,...,...,...,...
451,2021-01-05,119.51,161.63,3.853,6.2514
452,2021-01-04,113.71,151.27,3.574,5.4145
453,2021-01-03,93.37,146.33,3.437,5.4475
454,2021-01-02,85.81,136.60,3.199,4.8564


In [22]:
import plotly.express as px
fig = px.line(df, x="date", y=df.columns,
            #   hover_data={"date": "|%B %d, %Y"},
              title='custom tick labels')

fig.update_xaxes(
    dtick="M1")
    # tickformat="%b\n%Y")
fig.show()

In [23]:
import pandas as pd
## Group 1
grp1= ['AAVE','COMPOUND','SUSHI','UNISWAP']
grp1= ['aave','compound','sushi','uniswap']
grp2= ['CURVE']


aave = pd.read_csv('priceData/aave_price_data.csv')
compound = pd.read_csv('priceData/compound_price_data.csv')
sushi = pd.read_csv('priceData/sushi_price_data.csv')
uniswap = pd.read_csv('priceData/uniswap_price_data.csv')

date = aave['Date']
aave = aave['Price']
compound = compound['Price']
sushi = sushi['Price']
uniswap = uniswap['Price']
newdate = []

for i in date:
    newdate.append(mdy_to_ymd(i))
d = {'date':newdate,'aave':aave,'compound':compound,'sushi':sushi,'uniswap':uniswap}

df = pd.DataFrame(data=d)

In [28]:
# import some stuff
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

# # create some data
# df = pd.DataFrame()
# n = 50
# df["Time"] = np.arange(n)
# df["Linear-"] = np.arange(n)+np.random.rand(n)
# df["Linear+"] = np.arange(n)+np.random.rand(n)
# df["Log-"] = np.arange(n)+np.random.rand(n)
# df["Log+"] = np.arange(n)+np.random.rand(n)
# df.set_index("Time", inplace=True)

subfig = make_subplots(specs=[[{"secondary_y": True}]])

# create two independent figures with px.line each containing data from multiple columns
fig = px.line(df, x="date",y=['aave','compound'], render_mode="webgl")
fig2 = px.line(df, x="date",y=['sushi','uniswap'], render_mode="webgl")

fig2.update_traces(yaxis="y2")

    # tickformat="%b\n%Y")

subfig.add_traces(fig.data + fig2.data)
subfig.layout.xaxis.title="Time"
subfig.layout.yaxis.title="AAVE, COMPOUND"
subfig.layout.yaxis2.title="SUSHI, UNISWAP"
# recoloring is necessary otherwise lines from fig und fig2 would share each color
# e.g. Linear-, Log- = blue; Linear+, Log+ = red... we don't want this
subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))
subfig.update_xaxes(dtick="M1")

# subfig.update_layout(template='plotly_dark')
subfig.show()

In [ ]:
import plotly.graph_objects as go

import networkx as nx

G = nx.random_geometric_graph(200, 0.125)

In [30]:
from os import walk

In [54]:
f = []
for (dirpath, dirnames, filenames) in walk('sentimentalOutput'):
    f.extend(filenames)
    break
f

['discord-compound.json',
 'discord-aave.json',
 'discord-uniswap.json',
 'discord-grp2.json',
 'discord-grp1.json',
 'discord-curve.json',
 'discord-sushi.json']

In [55]:
import ast

In [65]:
tmp = ""
for i in f:
    arr = i.split('-')
    coin = arr[1].split('.')[0]
    option = arr[0] + " " + coin.upper()
    # print("option", option)
    tmp += option + ","
    # print("tmp", tmp)
    # tmp += "{'label': '" + i + "', 'value': '" + i + "'},"
# print(type(tmp))
tmp
tmp[:-2]
tmp[:-2].split(',')
# ast.literal_eval(tmp[:-1])

['discord COMPOUND',
 'discord AAVE',
 'discord UNISWAP',
 'discord GRP2',
 'discord GRP1',
 'discord CURVE',
 'discord SUSH']

In [33]:
for my_file in f:
    my_str += "{'label': '" + my_file + "', 'value': '" + my_file + "'},"
my_dict = dict(my_str)
my_dict

ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [77]:
f = []
for (dirpath, dirnames, filenames) in walk('priceData'):
    f.extend(filenames)
    break
tmp = []
for i in f:
    arr = i.split('_')
    x = arr[0]
    x = arr[0].upper()
    tmp.append({'label': x, 'value' : x})
tmp

[{'label': 'CURVE', 'value': 'CURVE'},
 {'label': 'SUSHI', 'value': 'SUSHI'},
 {'label': 'UNISWAP', 'value': 'UNISWAP'},
 {'label': 'COMPOUND', 'value': 'COMPOUND'},
 {'label': 'MAKER', 'value': 'MAKER'},
 {'label': 'AAVE', 'value': 'AAVE'}]

In [93]:
with open('sentimentalOutput/discord-curve.json') as json_file:
    data = json.load(json_file)
dict1 = {}
dict2 = {}
for i in data:
    arr = i.split("/")
    new_key = arr[1] + "-" + arr[0]
    dict1[new_key] = data[i]

    x = int(arr[0])
    if x == 12:
        new_x = 1
        new_key = str(int(arr[1]) + 1) + "-" + str(new_x)
    else:
        new_x = x + 1
        new_key = arr[1] + "-" + str(new_x)

    if len(new_key) == 6:
        arr = new_key.split('-')
        new_key = arr[0] + "-0" + arr[1] 
    dict2[new_key] = data[i]
dict2


{'2021-02': 0.18136256454388922,
 '2021-03': 0.15596198316867896,
 '2021-04': 0.17766406396989629,
 '2021-05': 0.20971195548489693,
 '2021-06': 0.17499569438581739,
 '2021-07': 0.14588408900786207,
 '2021-08': 0.16837303087586622,
 '2021-09': 0.16369584143023747,
 '2021-10': 0.15230753821824047,
 '2021-11': 0.15179191512712709,
 '2021-12': 0.14767883843064586,
 '2022-01': 0.14448721660748468,
 '2022-02': 0.14467588247296373,
 '2022-03': 0.15475518780215725,
 '2022-04': 0.1648167085427136}

In [94]:
dict1

{'2021-01': 0.18136256454388922,
 '2021-02': 0.15596198316867896,
 '2021-03': 0.17766406396989629,
 '2021-04': 0.20971195548489693,
 '2021-05': 0.17499569438581739,
 '2021-06': 0.14588408900786207,
 '2021-07': 0.16837303087586622,
 '2021-08': 0.16369584143023747,
 '2021-09': 0.15230753821824047,
 '2021-10': 0.15179191512712709,
 '2021-11': 0.14767883843064586,
 '2021-12': 0.14448721660748468,
 '2022-01': 0.14467588247296373,
 '2022-02': 0.15475518780215725,
 '2022-03': 0.1648167085427136}